In [1]:
import numpy as np
import pandas as pd
import matplotlib
import numpy as np
import sys
import pandas as pd
import psycopg2

In [2]:
engine = psycopg2.connect(
    database="cse6242team22db",
    user="Team22",
    password="vacayaway",
    host="cse6242-db.cvdup1gpakos.us-east-2.rds.amazonaws.com",
    port='5432'
)

In [3]:
cur = engine.cursor()

In [4]:
str_cmd = """SELECT nbc.*, nb.geom FROM neighborhoods nb join neighborhood_Crime_series nbc ON nb.gid = nbc.gid"""
cur.execute(str_cmd)
response = cur.fetchall()
pd.DataFrame(response)

str_cmd = """SELECT * FROM T"""
cur.execute(str_cmd)
response = cur.fetchall()
df_neighborhood_crime = pd.DataFrame(response, columns=[desc[0] for desc in cur.description])

str_cmd = """SELECT * FROM neighborhood_Crime_series"""
cur.execute(str_cmd)
response = cur.fetchall()
df_neighborhood_Crime_series = pd.DataFrame(response, columns=[desc[0] for desc in cur.description])


str_cmd = """SELECT * FROM neighborhoods"""
cur.execute(str_cmd)
response = cur.fetchall()
df_neighborhoods = pd.DataFrame(response, columns=[desc[0] for desc in cur.description])

str_cmd = """SELECT * FROM homeaway_v2"""
cur.execute(str_cmd)
response = cur.fetchall()
df_homeaway = pd.DataFrame(response, columns=[desc[0] for desc in cur.description])

In [5]:
df_neighborhood_crime

,gid,crime_score
0,1,66.307341
1,2,43.923431
2,3,47.083021
3,4,132.150403
4,5,11.043485
5,6,12.106519
6,7,3.902574
7,8,5.068972
8,9,53.128536
9,10,21.124712


In [ ]:
df_neighborhood_Crime_series

In [ ]:
df_neighborhoods['geom'][0]

In [ ]:
df_homeaway

In [ ]:
colnames = [desc[0] for desc in cur.description]

In [ ]:
t_lat = df_homeaway.iloc[0].latitude
t_long = df_homeaway.iloc[0].longitude

In [ ]:
df_homeaway.columns

In [ ]:
# str_cmd_block_1 = """
# SELECT crime_score
# FROM neighborhoods nb
# JOIN neighborhood_crime nbc ON nbc.gid = nb.gid
# WHERE st_contains(nb.atl_geom, st_setsrid(st_point(-84.3854511, 33.73753793), 4326))
# """

str_cmd_block_1 = """
SELECT crime_score
FROM neighborhoods nb
JOIN T nbc ON nbc.gid = nb.gid
WHERE st_contains(nb.atl_geom, st_setsrid(st_point"""

str_cmd_block_2 = str((t_long, t_lat))

str_cmd_block_3 = """, 4326))"""

str_cmd = str_cmd_block_1 + str_cmd_block_2 + str_cmd_block_3

cur.execute(str_cmd)
response = cur.fetchall()
test = pd.DataFrame(response, columns=[desc[0] for desc in cur.description])

In [ ]:
test.crime_score[0]

In [ ]:
str((t_long, t_lat))

This work can be functionalized as follows:

In [ ]:
def get_crime_score(t_lat, t_long, engine):    
    cur = engine.cursor()
    
    str_cmd_block_1 = """
    SELECT crime_score
    FROM neighborhoods nb
    JOIN T nbc ON nbc.gid = nb.gid
    WHERE st_contains(nb.atl_geom, st_setsrid(st_point"""
    
    str_cmd_block_2 = str((t_long, t_lat))

    str_cmd_block_3 = """, 4326))"""

    str_cmd = str_cmd_block_1 + str_cmd_block_2 + str_cmd_block_3
    
    #print(str_cmd)

    cur.execute(str_cmd)
    response = cur.fetchall()
    #print(response)
    if(len(response) == 0):
        crime_score = 513 # default to high value
    else:
        df_cs = pd.DataFrame(response, columns=[desc[0] for desc in cur.description])
        crime_score = df_cs.crime_score[0]
    
    return(crime_score)
    

In [ ]:
engine = psycopg2.connect(
    database="cse6242team22db",
    user="Team22",
    password="vacayaway",
    host="cse6242-db.cvdup1gpakos.us-east-2.rds.amazonaws.com",
    port='5432'
)

get_crime_score(t_lat, t_long, engine)

In [ ]:
df_homeaway['latitude'][0]

In [ ]:
df_homeaway['crime_score'] = 0 # instantiate values

import time
start = time.time()
for i in range(df_homeaway.shape[0]):
    #print(i)
    val = get_crime_score(df_homeaway['latitude'][i], df_homeaway['longitude'][i], engine)
    #print("Score:", val)
    df_homeaway['crime_score'][i] = val
    
time.time()-start

In [ ]:
df_homeaway['crime_score'][i] = get_crime_score(df_homeaway['latitude'][i], df_homeaway['longitude'][i], engine)

In [ ]:
df_homeaway['crime_score']

In [ ]:
response[0][0]